In [43]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import plotly.express as px
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

In [44]:
INPUT_SHAPE = (150,150,3)
BATCH_SIZE = 60
test_path ='../input/fruit-and-vegetable-image-recognition/test'
train_path ='../input/fruit-and-vegetable-image-recognition/train'

In [45]:
def get_generator(_training_dir, _test_dir):
    train_datagen = ImageDataGenerator(
        rescale=1/255.0, 
        featurewise_center=True,
        featurewise_std_normalization=True, 
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        shear_range=0.2,
        fill_mode='nearest'
    )
    test_datagen = ImageDataGenerator(
        rescale=1/255.0
    )
    train_generator = train_datagen.flow_from_directory(
        _training_dir,
        shuffle=True,
        target_size=(150,150),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    test_generator = test_datagen.flow_from_directory(
        _test_dir,
        shuffle=True,
        target_size=(150,150),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    
    return train_generator, test_generator

In [46]:
def plotter(history):
    #summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [47]:
def get_defined_label(x):
    label_dict = {}
    i=0
    for label in os.listdir(train_path):
        label_dict[i] = label
        i+=1
    for key, value in label_dict:
        if key == x:
            return value

In [48]:
def get_initialized_model(INPUT_SHAPE):
    base_model = InceptionResNetV2(
        input_shape=INPUT_SHAPE,
        weights='imagenet',
        include_top=False,
        pooling='avg'
    )
    base_model.trainable = False
    
    print('Base Model Ouput Shape = {}'.format(base_model.output_shape))
    last_output = base_model.output
    x = tf.keras.layers.Dropout(0.2)(last_output)
    x = tf.keras.layers.Dense(units=128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(units=128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(units = len(os.listdir(train_path)), activation='softmax')(x)
    
    model = tf.keras.Model( base_model.input, x) 

    model.compile(
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.9, centered=True), 
        loss = ['categorical_crossentropy'], 
        metrics = ['accuracy']
    )

    model_callback = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        min_delta=0.001, 
        patience=5, 
        verbose=1,
        mode='auto', 
        baseline=None, 
        restore_best_weights=True
    ), tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=1,
        min_delta=0.0000001,
        verbose=1 
    )]
    
    return model, model_callback

In [55]:
train_generator, test_generator = get_generator(train_path, test_path)
model, model_callbacks = get_initialized_model(INPUT_SHAPE)

In [56]:
history = model.fit_generator(train_generator, epochs=30, validation_data=test_generator)

In [57]:
score = model.evaluate_generator(test_generator)
plotter(history)
print('\nScore [ {} ] '.format(dict(zip(model.metrics_names, score))))